In [ ]:
# Load required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Load 2 data sets
app_rec = pd.read_csv(r'C:\Users\shaha\OneDrive\Documents\credit_score\application_record.csv')

cre_rec = pd.read_csv(r'C:\Users\shaha\OneDrive\Documents\credit_score\credit_record.csv')

# Data Exploration

In [ ]:
# Check the size of each data set
print(app_rec.shape)
print(cre_rec.shape)

In [ ]:
# To Evaluate Columns' Name:
app_rec.columns.to_list()

In [ ]:
# To Evaluate Data Type and Size:
app_rec.info()

In [ ]:
# Print first few raws of data and check them visually:
app_rec.head(3)

In [ ]:
# Create code for fields with object data type:
   # 'NAME_INCOME_TYPE',  'NAME_EDUCATION_TYPE',  'NAME_FAMILY_STATUS',  'NAME_HOUSING_TYPE'

print(app_rec.NAME_INCOME_TYPE.unique())
print(app_rec.NAME_EDUCATION_TYPE.unique())
print(app_rec.NAME_FAMILY_STATUS.unique())
print(app_rec.NAME_HOUSING_TYPE.unique())

# Create a for loop for visualization of frequency of items in each aforementioned columns
for var in ['NAME_EDUCATION_TYPE', 'NAME_INCOME_TYPE',   'NAME_FAMILY_STATUS',  'NAME_HOUSING_TYPE']:
    # countplt, ax = plt.subplots(figsize = (10,7))
    plt.rcParams["figure.figsize"] = [8, 5]
    plt.rcParams["figure.autolayout"] = True
    ax = sns.countplot(x =var, data = app_rec)
    ax.set_title(var,fontsize = 18, fontweight='bold' )
    ax.bar_label(ax.containers[0])
    ax.set_xlabel(var, fontsize = 15)
    ax.set_ylabel('count', fontsize = 15)
    plt.show()

In [ ]:
# It Seems we have imbalanced data. What will be the next step to address this situation and avoid consequential issues? Question from Saeed: What should I do in this situation?

In [ ]:
#Check the Numeric fields
app_rec.describe().T

In [ ]:
for var in ['CNT_CHILDREN']:  # It doenot have so much variables range!
    # countplt, ax = plt.subplots(figsize = (10,7))
    plt.rcParams["figure.figsize"] = [10, 7]
    plt.rcParams["figure.autolayout"] = True
    ax =  sns.countplot(x=var, data = app_rec)
    ax.set_title(var,fontsize = 18, fontweight='bold' )
    ax.bar_label(ax.containers[0])
    ax.set_xlabel(var, fontsize = 15)
    ax.set_ylabel('count', fontsize = 15)
    plt.show()

# My Graphs does not work correctly. Question from Saee: WHat I should do for th barplot?

In [ ]:
for var in ['AMT_INCOME_TOTAL',  'DAYS_EMPLOYED']:    # We have a wide range of variables in these columns!
    # countplt, ax = plt.subplots(figsize = (10,7))
    plt.rcParams["figure.figsize"] = [10, 7]
    plt.rcParams["figure.autolayout"] = True
    ax.set(yscale="log")
    ax =  sns.boxplot(app_rec[var])
    # ax.set(yscale="log")
    ax.set_title(var,fontsize = 18, fontweight='bold' )
    # ax.bar_label(ax.containers[0])
    ax.set_xlabel(var, fontsize = 15)
    ax.set_ylabel('count', fontsize = 15)
    plt.show();

In [ ]:
#Check Dublicate rows
app_rec.duplicated().sum()

We do not have any dublicated rows. However, we have dublicated IDs:

In [ ]:
app_rec.ID.duplicated().sum()

In [ ]:
app_rec_dup_ID = app_rec[app_rec['ID'].duplicated(keep=False) == True].sort_values(by=['ID']).reset_index(drop=True)
app_rec_dup_ID

Due to the sensitivity of the dublication in ID, as a key, I decided to drop all rows with dublicated IDs.(please see the Cleaning part)

In [ ]:
#check Missing Values
app_rec.isnull().sum()

In [ ]:
# Check the OCCUPATION_Type:

# print("Number of Missing Values:", app_rec.OCCUPATION_TYPE.isnull().sum())
# app_rec.OCCUPATION_TYPE.value_counts()

app_rec.OCCUPATION_TYPE.value_counts(dropna = False)  # The same as above but much more efficient!


# Data Cleaning

In [ ]:
# Change the data type of the 'ID' to string data type:
columns = ['ID']#,'FLAG_MOBIL', 'FLAG_WORK_PHONE', 'FLAG_PHONE', 'FLAG_EMAIL']
for column in columns:
    app_rec[column] = app_rec[column].astype(str)
    
print(app_rec["ID"].dtypes)

Note: It seems we have some Outliers in "AMT_INCOME_TOTAL". We need to do more research on it.

In [ ]:
# we have outliers in AMT_INCOME_TOTAL. We have 2 choices, either remove the outliers or creat buckets for values based on the 8 percentile groups. 
# The second option was selected and finally applied the one hot encoding to the buckets:
app_rec['AMT_INCOME_TOTAL_BUKET'], cut_bin = pd.qcut(app_rec['AMT_INCOME_TOTAL'], q = 8, labels = ['1','2','3','4','5','6','7','8'], retbins = True)
app_rec[['AMT_INCOME_TOTAL','AMT_INCOME_TOTAL_BUKET']]
# app_rec = pd.get_dummies(data=app_rec, columns=['AMT_INCOME_TOTAL_BUKET'])  # Do it later on in cleaning!

In [ ]:
# We can assign "Other" for null values. 
app_rec["OCCUPATION_TYPE"].fillna("Other", inplace = True)

In [ ]:
#Remove the duplicates values for IDs:
app_rec = app_rec.drop_duplicates(subset=['ID']).reset_index()

In [ ]:
# Replace any positive value of DAYS_EMPLOYED with 0:
app_rec['DAYS_EMPLOYED'] = app_rec['DAYS_EMPLOYED'].apply(lambda x: 0 if x > 0 else x)

In [ ]:
app_rec.shape

# Feature Engineering

In [ ]:
# Create month_employed from DAYS_EMPLOYED.
app_rec["MONTH_EMPLOYED"] = app_rec["DAYS_EMPLOYED"] /30
app_rec["MONTH_EMPLOYED"]  = app_rec["MONTH_EMPLOYED"].apply(np.ceil)
app_rec[["MONTH_EMPLOYED","DAYS_EMPLOYED"] ]

In [ ]:
# Create an employement flag fom DAYS_EMPLOYED.
# app_rec['FLAG_EMPLOYED2'] = app_rec['DAYS_EMPLOYED'].apply(lambda x: 1 if x > 0 else 0).astype(str)

# New: Bolean based on a condition instrad of code incell ablove.
app_rec["FLAG_EMPLOYED"] = app_rec['DAYS_EMPLOYED'] < 0 
app_rec.head(10)

In [ ]:
# Create Age based on DAYS_BIRTH.
app_rec['AGE'] = (app_rec['DAYS_BIRTH']/365).abs().apply(np.floor)
app_rec.hist(column='AGE');

In [ ]:
# Create 8 different bucket of Ages
bins = [0, 20, 25,30,45,55,60, 65, 70]
labels = [1,2,3,4,5,6,7,8]
app_rec['AGE_BUKET'] = pd.cut(app_rec['AGE'], bins,labels=labels)
app_rec['AGE_BUKET']

In [ ]:
# Create Count of Children bucket based on CNT_CHILDREN
app_rec.CNT_CHILDREN.value_counts()

In [ ]:
# Use OneHot_encoding to create binary values for 4 different fiels (see the above cell):
app_rec = pd.get_dummies(data=app_rec, columns=['NAME_INCOME_TYPE','NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS','NAME_HOUSING_TYPE'])#.astype(str)

In [ ]:
# # should be changed to number
# app_rec.CNT_CHILDREN_BUCKET = app_rec.CNT_CHILDREN
# def conditions(i): 
#     if i ==0 : return '0'
#     if i ==1 : return "1"
#     if i ==2 : return '2'
#     if i ==3 : return '3'
#     if i ==4 : return '4'
#     if i >= 5: return '5'

# app_rec['CNT_CHILDREN_BUCKET']=app_rec['CNT_CHILDREN'].apply(conditions)


# B - cre_rec Data Cleaning & Transforming

In [ ]:
print("Name of the columns:" ,cre_rec.columns)
print(cre_rec.shape)

In [ ]:
cre_rec.STATUS.unique()
# 0: 1-29 days past due 
# 1: 30-59 days past due 
# 2: 60-89 days overdue 
# 3: 90-119 days overdue 
# 4: 120-149 days overdue 
# 5: Overdue or bad debts, write-offs for more than 150 days 
# C: paid off that month 
# X: No loan for the month

In [ ]:
gouped_id = cre_rec.groupby('ID')
pivot_tb = cre_rec.pivot(index="ID", columns='MONTHS_BALANCE', values= 'STATUS')
pivot_tb["Open_Month"] = gouped_id["MONTHS_BALANCE"].min()
pivot_tb['End_Month'] = gouped_id['MONTHS_BALANCE'].max()
pivot_tb['ID'] = pivot_tb.index
pivot_tb = pivot_tb[["ID","Open_Month","End_Month"]]
pivot_tb["window"] = pivot_tb["End_Month"] - pivot_tb["Open_Month"]
pivot_tb.reset_index(drop = True, inplace = True)

credit = pd.merge(cre_rec, pivot_tb, on = 'ID', how = 'left')

In [ ]:
pivot_tb

In [ ]:
credit = credit[credit['window'] > 20]

credit['status'] = np.where((credit['STATUS'] == '2') | (credit['STATUS'] == '3' )| (credit['STATUS'] == '4' )| (credit['STATUS'] == '5'), 1, 0) # analyze > 60 days past due 

credit['status'] = credit['status'].astype(np.int8)

credit['month_on_book'] = credit['MONTHS_BALANCE'] - credit['Open_Month'] # calculate month on book: how many months after opening account

credit.sort_values(by = ['ID','month_on_book'], inplace = True)

####
denominator = pivot_tb.groupby(["Open_Month"]).agg({"ID":['count']})
denominator.reset_index(inplace=True)
denominator.columns = ['open_month','sta_sum']

In [ ]:
vintage = credit.groupby(['Open_Month','month_on_book']).agg({'ID': ['count']}) 
vintage.reset_index(inplace = True)
vintage.columns = ['open_month','month_on_book','sta_sum'] 
vintage['due_count'] = np.nan
vintage = vintage[['open_month','month_on_book','due_count']] # delete aggerate column
vintage = pd.merge(vintage, denominator, on = ['open_month'], how = 'left') # join sta_sum colun to vintage table
vintage

In [ ]:
credit[credit["status"]==1]

In [ ]:
for j in range(-60,1):
    ls = []
    for i in range(0,60):
        due = list(credit[(credit["status"]==1) & (credit["month_on_book"]==i) & (credit["Open_Month"]==j)]["ID"])
        ls.extend(due)
        vintage.loc[(vintage['month_on_book']==i) & (vintage['open_month']==j), "due_account"] = len(set(ls))

In [ ]:
vintage

In [ ]:
cre_rec.sort_values(["ID","MONTHS_BALANCE"],ascending=False,    inplace=False).head(10).set_index('ID')

In [ ]:
#Transpose the credit record 

In [ ]:
cre_rec = pd.DataFrame(cre_rec.groupby(["ID","STATUS"])["STATUS"].count()).rename(columns={'STATUS': 'Freq'}).reset_index()

In [ ]:
cre_rec

In [ ]:
cre_rec.pivot(index="ID", columns='MONTHS_BALANCE', values= 'STATUS')

In [ ]:
cre_rec.loc[cre_rec["ID"]==5001711]

In [ ]:
cre_rec.groupby(by=["ID"])

In [ ]:
cre_rec.ID.duplicated().count()

In [ ]:
cre_rec.shape

In [ ]:
cre_rec.STATUS.value_counts()

# C - Merge the Data Sets

In [ ]:
data = pd.merge(app_rec,cre_rec)

In [ ]:
data

In [ ]:
# Merge 2 datasets based on the id

In [ ]:
#Save the Cleaned Data

# D - Model Development

In [ ]:
# Reload the Cleaned Data

In [ ]:
#Reload the Cleaned Data

In [ ]:
#Load Required librarys
import sklearn 

In [ ]:
#train model 1

In [ ]:
#Test model 1

In [ ]:
#evaluate model 1

In [ ]:
pwd

In [ ]:
# test 2 for my project

Misc

In [ ]:
# sns.set_style("whitegrid")
sns.set(rc={"figure.figsize":(10, 10)})
sns.set_style("darkgrid", {"grid.color": ".6", "grid.linestyle": ":"})
ax = sns.boxplot( y=app_rec["AMT_INCOME_TOTAL"] )
ax.set(xlabel='Income', ylabel='Dollar Value')

